In [2]:
import numpy as np
import pptk
from pathlib import Path

from data_utils.MastersDataset import MastersDataset

from sklearn.ensemble import RandomForestClassifier

# Random Forests
## Get training data

In [3]:
data_path = Path('/home/luc/PycharmProjects/Pointnet_Pointnet2_pytorch/data/PatrickData/Church/MastersFormat/hand_selected_reversed')

train_data = MastersDataset('train', data_path, sample_all_points=True)
val_data = MastersDataset('validate', data_path, sample_all_points=True)

100%|██████████| 1/1 [00:00<00:00,  2.64it/s]


Sorting by x axis...3.04s


100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


Sorting by x axis...8.13s


Fill batches: 100%|██████████| 364/364 [00:43<00:00,  8.29it/s]


In [4]:
classifier = RandomForestClassifier(n_estimators=10, max_depth=30, min_samples_leaf=0.0001, n_jobs=-1, verbose=1)

classifier.fit(X=np.vstack(train_data.data_segment), y=np.hstack(train_data.labels_segment))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.0min finished


RandomForestClassifier(max_depth=30, min_samples_leaf=0.0001, n_estimators=10,
                       n_jobs=-1, verbose=1)

In [5]:
classifier.feature_importances_

array([0.10868859, 0.30054537, 0.54122266, 0.04954338])

In [6]:
classifier.n_features_


/home/luc/anaconda3/envs/Pointnet_Pointnet2_pytorch/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


4

In [7]:
classifier.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 0.0001,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 1,
 'warm_start': False}

In [8]:
preds = classifier.predict(np.vstack(val_data.data_segment))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    3.6s finished


In [9]:
# Evaluating the algorithm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
all_labels  = np.hstack(val_data.labels_segment)
print(f"confusion_matrix:\n{confusion_matrix(all_labels, preds)}")
print(f"classification_report:\n{classification_report(all_labels, preds)}")
print(f"accuracy score: {accuracy_score(all_labels, preds)}")
print(f"Feature importances: {classifier.feature_importances_}")

confusion_matrix:
[[14121823  1715961]
 [  276855   629809]]
classification_report:
              precision    recall  f1-score   support

           0       0.98      0.89      0.93  15837784
           1       0.27      0.69      0.39    906664

    accuracy                           0.88  16744448
   macro avg       0.62      0.79      0.66  16744448
weighted avg       0.94      0.88      0.90  16744448

accuracy score: 0.880986461900685
Feature importances: [0.10868859 0.30054537 0.54122266 0.04954338]
